In [1]:
import imblearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
from joblib import dump, load
import os

In [2]:
def fit_model(model, xs_train, xs_test, ys_train, ys_test, mode, method):
    if mode == 'save':
        model.fit(xs_train, ys_train.ravel())
        dump(model, os.path.join('models', method + '.joblib'))
    print(f"Rezultat trening skupa: {model.score(xs_train, ys_train):.3f}")
    print(f"Rezultat test skupa: {model.score(xs_test, ys_test):.3f}")


def prediction(model, x_train, x_test, y_train, y_test):
    y_train_predicted = model.predict(x_train)
    y_test_predicted = model.predict(x_test)
    print("Matrica kofuzije trening skupa:\n" + str(confusion_matrix(y_train, y_train_predicted)))
    print("Matrica kofuzije test skupa:\n" + str(confusion_matrix(y_test, y_test_predicted)))
    # print("Metrika: \n" + str(metrics.classification_report(y_test, y_test_predicted)))


# K nearest neighbours
def knn(x_train, x_test, y_train, y_test, non, mode, rtype):
    print("-----K najblizih suseda: " + rtype + " -----")
    start = time.time()
    if mode == 'load':
        model = load(os.path.join('models', 'knn_' + rtype + '.joblib'))
    else:
        model = KNeighborsClassifier(n_neighbors=non, weights='uniform')
    fit_model(model, x_train, x_test, y_train, y_test, mode, 'knn_' + rtype)
    prediction(model, x_train, x_test, y_train, y_test)
    print(f"Vreme izvrsavanja: {time.time()-start:.3f} \n")


# Decision tree classifier
def dtc(xs_train, xs_test, ys_train, ys_test, mode, rtype, criteria, depth):
    print("-----Drvo odlucivanja: " + rtype + " -----")
    start = time.time()
    if mode == 'load':
        model = load(os.path.join('models', 'dtc_' + rtype + '.joblib'))
    else:
        model = DecisionTreeClassifier(criterion=criteria, max_depth=depth)
    fit_model(model, xs_train, xs_test, ys_train, ys_test, mode, 'dtc_' + rtype)
    prediction(model, xs_train, xs_test, ys_train, ys_test)
    print(f"Vreme izvrsavanja: {time.time() - start:.3f} \n")
    

# Random forest classifier
def rfc(x_train, x_test, y_train, y_test, n_est, mode, rtype, maxd):
    print("-----Nasumicna suma: " + rtype + " -----")
    start = time.time()
    if mode == 'load':
        model = load(os.path.join('models', 'rfc_' + rtype + '.joblib'))
    else:
        model = RandomForestClassifier(n_estimators=n_est, max_depth=maxd, criterion='gini')
    # noinspection PyTypeChecker
    fit_model(model, x_train, x_test, y_train, y_test, mode, 'rfc_' + rtype)
    prediction(model, x_train, x_test, y_train, y_test)
    print(f"Vreme izvrsavanja: {time.time() - start:.3f} \n")

In [3]:
file = 'data_for_classification.csv'
df = pd.read_csv(file)

x = df.loc[:, df.columns != 'url']
x = x.loc[:, x.columns != 'class']
x = x.values[:, :]
y = df.values[:, -1]
y = y.astype('int')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

ratio = 1/2
rus = imblearn.under_sampling.RandomUnderSampler(random_state=0, sampling_strategy=ratio)
x_train_res, y_train_res = rus.fit_resample(x_train, y_train)
# x_test_res, y_test_res = rus.fit_resample(x_test, y_test)

smote = imblearn.over_sampling.SMOTE(random_state=0, k_neighbors=5)
x_train_res_smote, y_train_res_smote = smote.fit_resample(x_train, y_train)
# x_test_res_smote, y_test_res_smote = smote.fit_resample(x_test, y_test)

In [4]:
dtc(x_train, x_test, y_train, y_test, mode='save', rtype='NormalGiniNone', criteria='gini', depth=None)
dtc(x_train_res, x_test, y_train_res, y_test, mode='save', rtype='RatioGiniNone', criteria='gini', depth=None)
dtc(x_train_res_smote, x_test, y_train_res_smote, y_test, mode='save', rtype='SmoteGiniNone', criteria='gini', depth=None)

-----Drvo odlucivanja: NormalGiniNone -----
Rezultat trening skupa: 0.981
Rezultat test skupa: 0.928
Matrica kofuzije trening skupa:
[[ 43777   2867]
 [  2614 238768]]
Matrica kofuzije test skupa:
[[15650  4372]
 [ 4506 98912]]
Vreme izvrsavanja: 3.606 

-----Drvo odlucivanja: RatioGiniNone -----
Rezultat trening skupa: 0.976
Rezultat test skupa: 0.907
Matrica kofuzije trening skupa:
[[45165  1479]
 [ 1902 91386]]
Matrica kofuzije test skupa:
[[17033  2989]
 [ 8440 94978]]
Vreme izvrsavanja: 1.594 

-----Drvo odlucivanja: SmoteGiniNone -----
Rezultat trening skupa: 0.979
Rezultat test skupa: 0.924
Matrica kofuzije trening skupa:
[[236065   5317]
 [  4803 236579]]
Matrica kofuzije test skupa:
[[16134  3888]
 [ 5450 97968]]
Vreme izvrsavanja: 5.132 



In [5]:
dtc(x_train, x_test, y_train, y_test, mode='save', rtype='NormalEntropyNone', criteria='entropy', depth=None)
dtc(x_train_res, x_test, y_train_res, y_test, mode='save', rtype='RatioEntropyNone', criteria='entropy', depth=None)
dtc(x_train_res_smote, x_test, y_train_res_smote, y_test, mode='save', rtype='SmoteEntropyNone', criteria='entropy', depth=None)

-----Drvo odlucivanja: NormalEntropyNone -----
Rezultat trening skupa: 0.981
Rezultat test skupa: 0.930
Matrica kofuzije trening skupa:
[[ 43777   2867]
 [  2614 238768]]
Matrica kofuzije test skupa:
[[15660  4362]
 [ 4326 99092]]
Vreme izvrsavanja: 3.228 

-----Drvo odlucivanja: RatioEntropyNone -----
Rezultat trening skupa: 0.976
Rezultat test skupa: 0.908
Matrica kofuzije trening skupa:
[[45165  1479]
 [ 1902 91386]]
Matrica kofuzije test skupa:
[[17022  3000]
 [ 8367 95051]]
Vreme izvrsavanja: 1.520 

-----Drvo odlucivanja: SmoteEntropyNone -----
Rezultat trening skupa: 0.979
Rezultat test skupa: 0.925
Matrica kofuzije trening skupa:
[[236065   5317]
 [  4803 236579]]
Matrica kofuzije test skupa:
[[16115  3907]
 [ 5405 98013]]
Vreme izvrsavanja: 4.821 



In [6]:
dtc(x_train, x_test, y_train, y_test, mode='save', rtype='NormalGini5', criteria='gini', depth=5)
dtc(x_train_res, x_test, y_train_res, y_test, mode='save', rtype='RatioGini5', criteria='gini', depth=5)
dtc(x_train_res_smote, x_test, y_train_res_smote, y_test, mode='save', rtype='SmoteGini5', criteria='gini', depth=5)

-----Drvo odlucivanja: NormalGini5 -----
Rezultat trening skupa: 0.889
Rezultat test skupa: 0.890
Matrica kofuzije trening skupa:
[[ 30330  16314]
 [ 15669 225713]]
Matrica kofuzije test skupa:
[[13149  6873]
 [ 6685 96733]]
Vreme izvrsavanja: 1.866 

-----Drvo odlucivanja: RatioGini5 -----
Rezultat trening skupa: 0.844
Rezultat test skupa: 0.885
Matrica kofuzije trening skupa:
[[32309 14335]
 [ 7479 85809]]
Matrica kofuzije test skupa:
[[13988  6034]
 [ 8132 95286]]
Vreme izvrsavanja: 0.944 

-----Drvo odlucivanja: SmoteGini5 -----
Rezultat trening skupa: 0.846
Rezultat test skupa: 0.882
Matrica kofuzije trening skupa:
[[186877  54505]
 [ 19622 221760]]
Matrica kofuzije test skupa:
[[13829  6193]
 [ 8370 95048]]
Vreme izvrsavanja: 2.444 



In [7]:
 dtc(x_train, x_test, y_train, y_test, mode='save', rtype='NormalEntropy5', criteria='entropy', depth=5)
dtc(x_train_res, x_test, y_train_res, y_test, mode='save', rtype='RatioEntropy5', criteria='entropy', depth=5)
dtc(x_train_res_smote, x_test, y_train_res_smote, y_test, mode='save', rtype='SmoteEntropy5', criteria='entropy', depth=5)

-----Drvo odlucivanja: NormalEntropy5 -----
Rezultat trening skupa: 0.883
Rezultat test skupa: 0.884
Matrica kofuzije trening skupa:
[[ 30656  15988]
 [ 17592 223790]]
Matrica kofuzije test skupa:
[[13264  6758]
 [ 7514 95904]]
Vreme izvrsavanja: 1.878 

-----Drvo odlucivanja: RatioEntropy5 -----
Rezultat trening skupa: 0.841
Rezultat test skupa: 0.883
Matrica kofuzije trening skupa:
[[31797 14847]
 [ 7434 85854]]
Matrica kofuzije test skupa:
[[13743  6279]
 [ 8110 95308]]
Vreme izvrsavanja: 0.921 

-----Drvo odlucivanja: SmoteEntropy5 -----
Rezultat trening skupa: 0.850
Rezultat test skupa: 0.885
Matrica kofuzije trening skupa:
[[188415  52967]
 [ 19604 221778]]
Matrica kofuzije test skupa:
[[14071  5951]
 [ 8303 95115]]
Vreme izvrsavanja: 2.566 



In [8]:
rfc(x_train, x_test, y_train, y_test, n_est=10, mode='save', rtype='Normal10', maxd=None)
rfc(x_train_res, x_test, y_train_res, y_test, n_est=10, mode='save', rtype='Ratio10', maxd=None)
rfc(x_train_res_smote, x_test, y_train_res_smote, y_test, n_est=10, mode='save', rtype='Smote10', maxd=None)

-----Nasumicna suma: Normal10 -----
Rezultat trening skupa: 0.979
Rezultat test skupa: 0.940
Matrica kofuzije trening skupa:
[[ 42983   3661]
 [  2471 238911]]
Matrica kofuzije test skupa:
[[ 15817   4205]
 [  3160 100258]]
Vreme izvrsavanja: 7.196 

-----Nasumicna suma: Ratio10 -----
Rezultat trening skupa: 0.972
Rezultat test skupa: 0.924
Matrica kofuzije trening skupa:
[[44599  2045]
 [ 1906 91382]]
Matrica kofuzije test skupa:
[[17301  2721]
 [ 6718 96700]]
Vreme izvrsavanja: 3.212 

-----Nasumicna suma: Smote10 -----
Rezultat trening skupa: 0.978
Rezultat test skupa: 0.936
Matrica kofuzije trening skupa:
[[235516   5866]
 [  4923 236459]]
Matrica kofuzije test skupa:
[[16561  3461]
 [ 4498 98920]]
Vreme izvrsavanja: 10.714 



In [9]:
rfc(x_train, x_test, y_train, y_test, n_est=50, mode='save', rtype='Norma50', maxd=None)
rfc(x_train_res, x_test, y_train_res, y_test, n_est=50, mode='save', rtype='Ratio50', maxd=None)
rfc(x_train_res_smote, x_test, y_train_res_smote, y_test, n_est=50, mode='save', rtype='Smote50', maxd=None)

-----Nasumicna suma: Norma50 -----
Rezultat trening skupa: 0.981
Rezultat test skupa: 0.944
Matrica kofuzije trening skupa:
[[ 43257   3387]
 [  2129 239253]]
Matrica kofuzije test skupa:
[[ 15676   4346]
 [  2625 100793]]
Vreme izvrsavanja: 31.223 

-----Nasumicna suma: Ratio50 -----
Rezultat trening skupa: 0.976
Rezultat test skupa: 0.929
Matrica kofuzije trening skupa:
[[44857  1787]
 [ 1610 91678]]
Matrica kofuzije test skupa:
[[17179  2843]
 [ 5958 97460]]
Vreme izvrsavanja: 13.408 

-----Nasumicna suma: Smote50 -----
Rezultat trening skupa: 0.979
Rezultat test skupa: 0.940
Matrica kofuzije trening skupa:
[[235698   5684]
 [  4462 236920]]
Matrica kofuzije test skupa:
[[16480  3542]
 [ 3921 99497]]
Vreme izvrsavanja: 51.687 



In [10]:
rfc(x_train, x_test, y_train, y_test, n_est=100, mode='save', rtype='Normal100', maxd=None)
rfc(x_train_res, x_test, y_train_res, y_test, n_est=100, mode='save', rtype='Ratio100', maxd=None)
rfc(x_train_res_smote, x_test, y_train_res_smote, y_test, n_est=100, mode='save', rtype='Smote100', maxd=None)

-----Nasumicna suma: Normal100 -----
Rezultat trening skupa: 0.981
Rezultat test skupa: 0.944
Matrica kofuzije trening skupa:
[[ 43239   3405]
 [  2081 239301]]
Matrica kofuzije test skupa:
[[ 15688   4334]
 [  2583 100835]]
Vreme izvrsavanja: 59.044 

-----Nasumicna suma: Ratio100 -----
Rezultat trening skupa: 0.976
Rezultat test skupa: 0.931
Matrica kofuzije trening skupa:
[[44870  1774]
 [ 1609 91679]]
Matrica kofuzije test skupa:
[[17245  2777]
 [ 5794 97624]]
Vreme izvrsavanja: 24.856 

-----Nasumicna suma: Smote100 -----
Rezultat trening skupa: 0.979
Rezultat test skupa: 0.940
Matrica kofuzije trening skupa:
[[235753   5629]
 [  4495 236887]]
Matrica kofuzije test skupa:
[[16463  3559]
 [ 3843 99575]]
Vreme izvrsavanja: 99.909 



In [11]:
knn(x_train, x_test, y_train, y_test, non=3, mode='save', rtype='Normal3')
knn(x_train_res, x_test, y_train_res, y_test, non=3, mode='save', rtype='Ratio3')
knn(x_train_res_smote, x_test, y_train_res_smote, y_test, non=3, mode='save', rtype='Smote3')

-----K najblizih suseda: Normal3 -----
Rezultat trening skupa: 0.948
Rezultat test skupa: 0.919
Matrica kofuzije trening skupa:
[[ 37452   9192]
 [  5723 235659]]
Matrica kofuzije test skupa:
[[14174  5848]
 [ 4108 99310]]
Vreme izvrsavanja: 86.218 

-----K najblizih suseda: Ratio3 -----
Rezultat trening skupa: 0.898
Rezultat test skupa: 0.854
Matrica kofuzije trening skupa:
[[42942  3702]
 [10596 82692]]
Matrica kofuzije test skupa:
[[17147  2875]
 [15192 88226]]
Vreme izvrsavanja: 43.759 

-----K najblizih suseda: Smote3 -----
Rezultat trening skupa: 0.949
Rezultat test skupa: 0.902
Matrica kofuzije trening skupa:
[[229066  12316]
 [ 12372 229010]]
Matrica kofuzije test skupa:
[[16287  3735]
 [ 8362 95056]]
Vreme izvrsavanja: 284.925 



In [12]:
knn(x_train, x_test, y_train, y_test, non=5, mode='save', rtype='Normal5')
knn(x_train_res, x_test, y_train_res, y_test, non=5, mode='save', rtype='Ratio5')
knn(x_train_res_smote, x_test, y_train_res_smote, y_test, non=5, mode='save', rtype='Smote5')

-----K najblizih suseda: Normal5 -----
Rezultat trening skupa: 0.938
Rezultat test skupa: 0.918
Matrica kofuzije trening skupa:
[[ 34941  11703]
 [  6185 235197]]
Matrica kofuzije test skupa:
[[13656  6366]
 [ 3769 99649]]
Vreme izvrsavanja: 92.322 

-----K najblizih suseda: Ratio5 -----
Rezultat trening skupa: 0.884
Rezultat test skupa: 0.857
Matrica kofuzije trening skupa:
[[41399  5245]
 [10951 82337]]
Matrica kofuzije test skupa:
[[16878  3144]
 [14464 88954]]
Vreme izvrsavanja: 49.346 

-----K najblizih suseda: Smote5 -----
Rezultat trening skupa: 0.939
Rezultat test skupa: 0.897
Matrica kofuzije trening skupa:
[[228032  13350]
 [ 16191 225191]]
Matrica kofuzije test skupa:
[[16579  3443]
 [ 9234 94184]]
Vreme izvrsavanja: 298.978 



In [13]:
 knn(x_train, x_test, y_train, y_test, non=10, mode='save', rtype='Normal10')
knn(x_train_res, x_test, y_train_res, y_test, non=10, mode='save', rtype='Ratio10')
knn(x_train_res_smote, x_test, y_train_res_smote, y_test, non=10, mode='save', rtype='Smote10')

-----K najblizih suseda: Normal10 -----
Rezultat trening skupa: 0.927
Rezultat test skupa: 0.916
Matrica kofuzije trening skupa:
[[ 33656  12988]
 [  8147 233235]]
Matrica kofuzije test skupa:
[[13645  6377]
 [ 4045 99373]]
Vreme izvrsavanja: 103.265 

-----K najblizih suseda: Ratio10 -----
Rezultat trening skupa: 0.869
Rezultat test skupa: 0.853
Matrica kofuzije trening skupa:
[[40810  5834]
 [12567 80721]]
Matrica kofuzije test skupa:
[[17017  3005]
 [15141 88277]]
Vreme izvrsavanja: 53.749 

-----K najblizih suseda: Smote10 -----
Rezultat trening skupa: 0.922
Rezultat test skupa: 0.878
Matrica kofuzije trening skupa:
[[229483  11899]
 [ 25900 215482]]
Matrica kofuzije test skupa:
[[17409  2613]
 [12392 91026]]
Vreme izvrsavanja: 324.954 

